# ML2025 Homework 1 - Retrieval Augmented Generation with Agents

## Environment Setup

First, we will mount your own Google Drive and change the working directory.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Change the working directory to somewhere in your Google Drive.
# You could check the path by right clicking on the folder.
# %cd [change to the directory you prefer]
%cd /content/drive/MyDrive/QA

/content/drive/MyDrive/QA


In this section, we install the necessary python packages and download model weights of the quantized version of LLaMA 3.1 8B. Also, download the dataset. Note that the model weight is around 8GB. If you are using your Google Drive as the working directory, make sure you have enough space for the model.

In [ ]:
!python3 -m pip install --no-cache-dir llama-cpp-python==0.3.4 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122
!python3 -m pip install googlesearch-python bs4 charset-normalizer requests-html lxml_html_clean

from pathlib import Path
if not Path('./Meta-Llama-3.1-8B-Instruct-Q8_0.gguf').exists():
    !wget https://huggingface.co/bartowski/Meta-Llama-3.1-8B-Instruct-GGUF/resolve/main/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf
if not Path('./public.txt').exists():
    !wget https://www.csie.ntu.edu.tw/~ulin/public.txt
if not Path('./private.txt').exists():
    !wget https://www.csie.ntu.edu.tw/~ulin/private.txt

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu122
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.2/445.2 MB 135.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.5.0
    Uninstalling urllib3-2.5.0:
      Successfully uninstalled urllib3-2.5.0
ERROR: pip's de

In [ ]:
import torch
if not torch.cuda.is_available():
    raise Exception('You are not using the GPU runtime. Change it first or you will suffer from the super slow inference speed!')
else:
    print('You are good to go!')

You are good to go!


## Prepare the LLM and LLM utility function

By default, we will use the quantized version of LLaMA 3.1 8B. you can get full marks on this homework by using the provided LLM and LLM utility function. You can also try out different LLM models.

In the following code block, we will load the downloaded LLM model weights onto the GPU first.
Then, we implemented the generate_response() function so that you can get the generated response from the LLM model more easily.

You can ignore "llama_new_context_with_model: n_ctx_per_seq (16384) < n_ctx_train (131072) -- the full capacity of the model will not be utilized" warning.

In [ ]:
from llama_cpp import Llama

# Load the model onto GPU
llama3 = Llama(
    "./Meta-Llama-3.1-8B-Instruct-Q8_0.gguf",
    verbose=False,
    n_gpu_layers=-1,
    n_ctx=16384,    # This argument is how many tokens the model can take. The longer the better, but it will consume more memory. 16384 is a proper value for a GPU with 16GB VRAM.
)

def generate_response(_model: Llama, _messages: str) -> str:
    '''
    This function will inference the model with given messages.
    '''
    _output = _model.create_chat_completion(
        _messages,
        stop=["<|eot_id|>", "<|end_of_text|>"],
        max_tokens=512,    # This argument is how many tokens the model can generate, you can change it and observe the differences.
        temperature=0,      # This argument is the randomness of the model. 0 means no randomness. You will get the same result with the same input every time. You can try to set it to different values.
        repeat_penalty=2.0,
    )["choices"][0]["message"]["content"]
    return _output

llama_new_context_with_model: n_ctx_per_seq (16384) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


## Search Tool

The TA has implemented a search tool for you to search certain keywords using Google Search. You can use this tool to search for the relevant **web pages** for the given question. The search tool can be integrated in the following sections.

In [ ]:
from typing import List
from googlesearch import search as _search
from bs4 import BeautifulSoup
from charset_normalizer import detect
import asyncio
from requests_html import AsyncHTMLSession
import urllib3
urllib3.disable_warnings()

async def worker(s:AsyncHTMLSession, url:str):
    try:
        header_response = await asyncio.wait_for(s.head(url, verify=False), timeout=10)
        if 'text/html' not in header_response.headers.get('Content-Type', ''):
            return None
        r = await asyncio.wait_for(s.get(url, verify=False), timeout=10)
        return r.text
    except:
        return None

async def get_htmls(urls):
    session = AsyncHTMLSession()
    tasks = (worker(session, url) for url in urls)
    return await asyncio.gather(*tasks)

async def search(keyword: str, n_results: int=3) -> List[str]:
    '''
    This function will search the keyword and return the text content in the first n_results web pages.

    Warning: You may suffer from HTTP 429 errors if you search too many times in a period of time. This is unavoidable and you should take your own risk if you want to try search more results at once.
    The rate limit is not explicitly announced by Google, hence there's not much we can do except for changing the IP or wait until Google unban you (we don't know how long the penalty will last either).
    '''
    keyword = keyword[:100]
    # First, search the keyword and get the results. Also, get 2 times more results in case some of them are invalid.
    results = list(_search(keyword, n_results * 2, lang="zh", unique=True))
    # Then, get the HTML from the results. Also, the helper function will filter out the non-HTML urls.
    results = await get_htmls(results)
    # Filter out the None values.
    results = [x for x in results if x is not None]
    # Parse the HTML.
    results = [BeautifulSoup(x, 'html.parser') for x in results]
    # Get the text from the HTML and remove the spaces. Also, filter out the non-utf-8 encoding.
    results = [''.join(x.get_text().split()) for x in results if detect(x.encode()).get('encoding') == 'utf-8']
    # Return the first n results.
    return results[:n_results]

## Test the LLM inference pipeline

In [ ]:
# You can try out different questions here.
test_question='請問誰是 Taylor Swift？'

messages = [
    {"role": "system", "content": "你是 LLaMA-3.1-8B，是用來回答問題的 AI。使用中文時只會使用繁體中文來回問題。"},    # System prompt
    {"role": "user", "content": test_question}, # User prompt
]

print(generate_response(llama3, messages))

泰勒絲（Taylor Swift）是一位美國歌手、詞曲作家和音樂製作人。她出生於1989年，來自田納西州。她的音乐风格从乡村摇滚发展到流行搖擺，並且她被誉为当代最成功的女艺人的之一。

泰勒絲早期在鄉郊小鎮演唱會時開始發展音樂事業，她推出了多張專輯，包括《Taylor Swift》、《Fearless》，以及後來更為知名的大熱作如 《1989》（2014年）、_reputation（）和 _Lover （）。她的歌曲經常探討愛情、友誼及自我成長等主題。

泰勒絲獲得了許多獎項，包括13座格萊美奖，並且是史上最快達到百萬銷量的女藝人之一。


In [ ]:
# 测试Search Tool
async def main():
    results = await search("第一代 iPhone 是由哪位蘋果 CEO 發表？", n_results=3)
    for i, text in enumerate(results):
        print(f"--- 第{i+1}篇 ---")
        print(text[:300] + "...\n")

await main()


--- 第1篇 ---
iPhone(第一代)-维基百科，自由的百科全书跳转到内容主菜单主菜单移至侧栏隐藏导航首页分类索引特色内容新闻动态最近更改随机条目帮助帮助维基社群方针与指引互助客栈知识问答字词转换IRC即时聊天联络我们关于维基百科特殊页面搜索搜索外观资助维基百科创建账号登录个人工具资助维基百科创建账号登录目录移至侧栏隐藏序言1历史开关历史子章节1.1研发1.2发售1.3发售后2设计3系列特点和功能4软件5评价6各代iPhone型号的时间轴7参考资料开关目录iPhone(第一代)48种语言ÆngliscالعربيةتۆرکجهবাংলাCatalàکوردیČeštinaDanskDeutschΕλληνικ...

--- 第2篇 ---
iPhone-维基百科，自由的百科全书跳转到内容主菜单主菜单移至侧栏隐藏导航首页分类索引特色内容新闻动态最近更改随机条目帮助帮助维基社群方针与指引互助客栈知识问答字词转换IRC即时聊天联络我们关于维基百科特殊页面搜索搜索外观资助维基百科创建账号登录个人工具资助维基百科创建账号登录目录移至侧栏隐藏序言1歷史开关歷史子章节1.1研发阶段1.2iPhone上市2型號开关型號子章节2.1在售设备2.2停售裝置3硬件开关硬件子章节3.1螢幕與按键3.2传感器3.2.1距离传感器3.2.2光线传感器3.2.3磁传感器3.2.4重力传感器3.2.5液體接觸感應器3.2.6迴轉儀感應器3.2.7无线电传感器3...

--- 第3篇 ---
歷史上的今天》1月9日──賈伯斯用手機改變歷史第一代iPhone問世首頁新聞股市運動TV汽機車購物中心拍賣氣象Yahoo國際App下載Yahoo奇摩新聞搜尋登入熱搜杜詩梅嗆律師台版紅姐排風扇邱正雄辭世飯店比價劉宇寧保險箱金庫大谷翔平日本關稅國光客運焦點即時大罷免風暴娛樂影劇國際政治社會地方財經運動玩樂品味健康遊戲3CFun暑假更多新聞專輯焦點每日Yahoo焦點Yahoo補助通Yahoo畫重點一周大事Yahoo獨家授權熱門新聞天氣民調Yahoo國際通話題PK小測驗合作媒體新聞專輯即時娛樂即時國際即時政治即時社會即時財經即時生活即時運動即時科技即時健康即時大罷免風暴罷免名單一次看罷免投票注意事項關...



## Agents

The TA has implemented the Agent class for you. You can use this class to create agents that can interact with the LLM model. The Agent class has the following attributes and methods:
- Attributes:
    - role_description: The role of the agent. For example, if you want this agent to be a history expert, you can set the role_description to "You are a history expert. You will only answer questions based on what really happened in the past. Do not generate any answer if you don't have reliable sources.".
    - task_description: The task of the agent. For example, if you want this agent to answer questions only in yes/no, you can set the task_description to "Please answer the following question in yes/no. Explanations are not needed."
    - llm: Just an indicator of the LLM model used by the agent.
- Method:
    - inference: This method takes a message as input and returns the generated response from the LLM model. The message will first be formatted into proper input for the LLM model. (This is where you can set some global instructions like "Please speak in a polite manner" or "Please provide a detailed explanation".) The generated response will be returned as the output.

In [ ]:
class LLMAgent():
    def __init__(self, role_description: str, task_description: str, llm:str="bartowski/Meta-Llama-3.1-8B-Instruct-GGUF"):
        self.role_description = role_description   # Role means who this agent should act like. e.g. the history expert, the manager......
        self.task_description = task_description    # Task description instructs what task should this agent solve.
        self.llm = llm  # LLM indicates which LLM backend this agent is using.
    def inference(self, message:str) -> str:
        if self.llm == 'bartowski/Meta-Llama-3.1-8B-Instruct-GGUF': # If using the default one.
            # TODO: Design the system prompt and user prompt here.
            # Format the messsages first.
            messages = [
                {"role": "system", "content": f"{self.role_description}"},  # Hint: you may want the agents to speak Traditional Chinese only.
                {"role": "user", "content": f"{self.task_description}\n{message}"}, # Hint: you may want the agents to clearly distinguish the task descriptions and the user messages. A proper seperation text rather than a simple line break is recommended.
            ]
            return generate_response(llama3, messages)
        else:
            # TODO: If you want to use LLMs other than the given one, please implement the inference part on your own.
            return ""

TODO: Design the role description and task description for each agent.

In [ ]:
# TODO: Design the role and task description for each agent.

# This agent may help you filter out the irrelevant parts in question descriptions.


question_extraction_agent = LLMAgent(
    role_description=(
        "你是一位專業的問題分析師，擅長從冗長、含混或背景繁複的敘述中，"
        "準確抽取出最核心、需要被解答的單一句完整問題。你只使用繁體中文回答。"
    ),
    task_description=(
        "請從下列敘述中，**萃取出一句內容完整、具體明確、可獨立理解的「疑問句」**，這句話必須清楚地表達使用者真正想知道的事情。\n\n"
        "請注意：\n"
        "- 僅輸出一句**具備問句語氣的問題句**，不可回述陳述句或背景敘述\n"
        "- **必須包含問詞**，如「是誰」、「什麼」、「哪裡」、「為何」、「怎麼做」等\n"
        "- 保留對理解問題有幫助的關鍵資訊（如主體、情境、時間）\n"
        "- 不得簡化成失去意義的泛泛提問；也不能只保留背景、忽略真正的提問\n"
        "- 禁止任何說明、標點或多句輸出，僅輸出一句能獨立理解的問題\n\n"
    )
)


# This agent may help you extract the keywords in a question so that the search tool can find more accurate results.
keyword_extraction_agent = LLMAgent(
    role_description=(
        "你是一位專業的搜尋關鍵字萃取專家，擅長從問題中提取出最適合用來搜尋的具體關鍵詞或短語。"
        "你只會用繁體中文回答。"
    ),
    task_description=(
        "請從下列問題中，**萃取出最能提升搜尋精準度的名詞或短語**，"
        "應包含有辨識力的具體資訊，例如：人名、課程名稱、機構、主題、時間、內容編號等。\n\n"
        "請避免：\n"
        "- 過於籠統的詞（如：問題、資料、內容）\n"
        "- 非名詞（如：想知道、了解、請問）\n\n"
        "請直接輸出以「逗號」分隔的關鍵字，不要加任何說明或多餘標點。\n\n"
        "【範例 1】\n"
        "輸入：李宏毅在2023年春季的《機器學習》課程中，第15個作業是什麼？\n"
        "輸出：李宏毅,2023年春季,機器學習,第15個作業\n\n"
        "【範例 2】\n"
        "輸入：請推薦台大電機系學生在大三可以選的通識好課。\n"
        "輸出：台大,電機系,大三,通識課,推薦課程\n\n"
        "【範例 3】\n"
        "輸入：為什麼貓會在你打電腦的時候跳到鍵盤上？\n"
        "輸出：貓,鍵盤行為,人貓互動,電腦\n"
    )
)



# This agent is the core component that answers the question.
# qa_agent = LLMAgent(
#     role_description=(
#         "你是一位專精於簡短問答的 AI 助理，只使用繁體中文回答問題。"
#         "你只能根據提供的資料回答，不得加入個人解釋或描述。"
#     ),
#     task_description=(
#         "請根據下列資料回答問題，並嚴格遵守以下格式與規則：\n\n"
#         "【回答格式】\n"
#         "只能回答一句完整句子，開頭為：「答：」，禁止其他內容。\n"
#         "若資料中沒有答案，請回答：「答：資料中未提及。」\n\n"
#         "【嚴格限制】\n"
#         "- 只能回答一句，禁止多句、禁止補充\n"
#         "- 禁止使用任何背景敘述、來源引用或感想\n"
#         "- 嚴禁重述資料內容，只能輸出答案句\n\n"
#         "【範例】\n"
#         "問題：第一代 iPhone 是由哪位蘋果 CEO 發表？\n"
#         "答：第一代 iPhone 是由 Steve Jobs（史提夫賈伯斯）發表。\n\n"
#         "請根據資料回答以下問題："
#     )
# )
qa_agent = LLMAgent(
    role_description="你是极简回答器，仅用繁体中文。",
    task_description=(
        "请用一句繁体中文回答，开头固定为「答：」。\n"
        "无答案时回复「答：资料中未提及。」"
    )
)


## RAG pipeline

TODO: Implement the RAG pipeline.

Please refer to the homework description slides for hints.

Also, there might be more heuristics (e.g. classifying the questions based on their lengths, determining if the question need a search or not, reconfirm the answer before returning it to the user......) that are not shown in the flow charts. You can use your creativity to come up with a better solution!

- Naive approach (simple baseline)

    ![](https://www.csie.ntu.edu.tw/~ulin/naive.png)

- Naive RAG approach (medium baseline)

    ![](https://www.csie.ntu.edu.tw/~ulin/naive_rag.png)

- RAG with agents (strong baseline)

    ![](https://www.csie.ntu.edu.tw/~ulin/rag_agent.png)

In [ ]:
# async def pipeline(question: str) -> str:
#     # TODO: Implement your pipeline.
#     # Currently, it only feeds the question directly to the LLM.
#     # You may want to get the final results through multiple inferences.
#     # Just a quick reminder, make sure your input length is within the limit of the model context window (16384 tokens), you may want to truncate some excessive texts.
#     return qa_agent.inference(question)

In [ ]:
question = "卑南族是位在臺東平原的一個原住民族，以驍勇善戰、擅長巫術聞名，曾經統治整個臺東平原。相傳卑南族的祖先發源自 ruvuwa'an，該地位於現今的哪個行政區劃？"
core_question = question_extraction_agent.inference(question)

In [ ]:
core_question

'該地位於現今的哪個行政區劃？'

In [ ]:
async def pipeline(question: str) -> str:
    # 问题抽取
    core_question = question_extraction_agent.inference(question)

    # 关键词抽取
    # keywords = keyword_extraction_agent.inference(core_question)

    # 抓取网页资料
    # search_results = await search(keywords, n_results=3)
    search_results = await search(core_question, n_results=3)

    # 限制每篇网页内容最大长度（例如 1500 字符）
    max_chars_per_doc = 3000

    trimmed_results = [
      f"[資料{i+1}]\n{text[:max_chars_per_doc]}" for i, text in enumerate(search_results)
      ]


    # 拼接为 web_content
    web_content = "\n".join(trimmed_results)

    # 构建提示词
    combined_prompt = f"""
    【任務說明】
    你是一位知識嚴謹的專家，請根據下方的網頁資料來回答問題。

    【網頁資料】
    {web_content}

    【使用者問題】
    {core_question}

    請用繁體中文詳細回答，並請不要引用網頁以外的資訊。
    """
    # 使用agent模型推理
    agent = LLMAgent(
      role_description="你是知識型 AI 助理，請使用提供的資料來回答問題。",
      task_description="任務：閱讀資料並根據問題作出推理與解釋。",
    )
    response = agent.inference(combined_prompt)
    return response

## Answer the questions using your pipeline!

Since Colab has usage limit, you might encounter the disconnections. The following code will save your answer for each question. If you have mounted your Google Drive as instructed, you can just rerun the whole notebook to continue your process.

In [ ]:
import pandas as pd
df = pd.read_csv('./qa.csv', encoding='utf-8')

In [ ]:
df.head()

,问题,答案
0,校歌為學校（包括小學、中學、大學等）宣告或者規定的代表該校的歌曲。用於體現該校的治學理念、辦...,「虎山雄風飛揚」是「光華國小」學校的校歌歌詞。
1,2025年初，NCC透過行政命令，規定民眾如果透過境外郵購無線鍵盤、滑鼠、藍芽耳機..等自用...,每案一律加收審查費750元。
2,第一代 iPhone 是由哪位蘋果 CEO 發表？,第一代 iPhone 是由Steve Jobs（史提夫賈伯斯）發表。
3,台灣大學進階英文免修申請規定中，托福網路測驗 TOEFL iBT 要達到多少分才能申請？,台灣大學進階英文免修申請規定中，托福網路測驗 TOEFL iBT 72分才能申請
4,Rugby Union 中觸地 try 可得幾分？,Rugby Union 中觸地 try 可得5分


In [ ]:
from pathlib import Path

# PREFIX = "test" # simple baseline
# PREFIX = "rag" # medium baseline
PREFIX = "system" # strong baseline

for idx, row in df.iterrows():
    question = row['问题']
    # answer   = row['答案']
    # print(f"{idx + 1} | 问题: {question} | 答案: {answer}")
    if Path(f"./{PREFIX}_{idx + 1}.txt").exists():
            continue
    answer = await pipeline(question)
    answer = answer.replace('\n',' ')
    print(idx + 1, answer)
    with open(f'./{PREFIX}_{idx + 1}.txt', 'w') as output_f:
        print(answer, file=output_f)

67 根據資料1，2024年世界棒球12強賽的冠軍是中華台北代表隊。
68 根據提供的資料，中國四大奇書是指《水滸傳》、《三國演義》，以及另外兩本小說。然而，這裡有個問題：在明代時期，被稱為「four 大 奇 曲」的小説包括了 《金瓶梅 》和西遊記，但是在清朝後來被取而替換的是《紅樓夢》。  因此，根據資料，我們可以得出以下的答案：  中國四大奇書是指： 1. 水滸傳 2._三國演義_ 3.._金瓶梅》 4 ._西遊記_  然而，這裡有個問題：在清朝後來被取而替換的是《紅樓夢》。
69 根據資料1，子時是從23點到01時間。因此，如果用24小时制表示，那就是：  * 子初：00:0000 （夜半） 至： 丑正 02 :  (鸡鸣)  所以，在12小时和20四个时间段中，我们可以看到，从晚上11点开始，子時是从23點到01時間。
70 根據提供的資料，避免錯過時限來完成任務的是Deadlinescheduler（截止日期排程器）。這種演算法基於EarliestDeadlineFirst (EDF) 的原理，即將優先權給予那些最早需要結束時間點(task deadline）的程序。
71 根據《刀劍神域》原作中的資料，代號「C8763」是由黑雪姬持有。
72 根據資料1中的描述，「柴城」位於現今的屏東車市，是恆春半島最早開發的地方。
73 根據資料中提供的資訊，若要使用 A100 高級 GPU，在 Google Colab 的訂閱制 中需要購買 "Colaboratory Pro+" 計畫。
74 根據資料1，李宏毅老師是電機工程學系教授，他開設的「生成式人工智慧導論」課程屬於台大文學院特別邀請外部教員為其跨域選修。
75 根據網頁資料，我們無法直接找到關於雪江學生簽減免申請書的相關資訊。然而，基礎上來說，大三（通常指的是大學三年級）是大多數國家教育系統中的一個階段。  如果我們假設問題與一般的情況有所相似，那麼在許多少種情形下，一般學生需要達到一定的成績標準才不用簽減免申請書。然而，這些資訊並未出現在網頁資料上，因此無法給予具體答案。  如果你能夠提供更多關於雪江大學或相關政策的情況，我們可能會有更好的回答能力，幫助您解答這個問題！
76 根據資料1，Neuro-sama 的最初 Live2D 模型是使用 "桃瀨日和／ 桃瀬ひより" 這個角色。
77 根據資料1和2，

In [ ]:
# Combine the results into one file.
with open(f'./{PREFIX}.txt', 'w') as output_f:
    for id in range(1,91):
        with open(f'./{PREFIX}_{id}.txt', 'r') as input_f:
            answer = input_f.readline().strip()
            print(answer, file=output_f)

## Evaluation
Using gemini api to evaluate the correctness of answers.

In [ ]:
# Check model list
import google.generativeai as genai

GEMINI_API_KEY=""
genai.configure(api_key=GEMINI_API_KEY)

models = genai.list_models()
for m in models:
    print(m.name, m.supported_generation_methods)


models/embedding-gecko-001 ['embedText', 'countTextTokens']
models/gemini-1.0-pro-vision-latest ['generateContent', 'countTokens']
models/gemini-pro-vision ['generateContent', 'countTokens']
models/gemini-1.5-pro-latest ['generateContent', 'countTokens']
models/gemini-1.5-pro-002 ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-pro ['generateContent', 'countTokens']
models/gemini-1.5-flash-latest ['generateContent', 'countTokens']
models/gemini-1.5-flash ['generateContent', 'countTokens']
models/gemini-1.5-flash-002 ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-flash-8b ['createCachedContent', 'generateContent', 'countTokens']
models/gemini-1.5-flash-8b-001 ['createCachedContent', 'generateContent', 'countTokens']
models/gemini-1.5-flash-8b-latest ['createCachedContent', 'generateContent', 'countTokens']
models/gemini-2.5-pro-preview-03-25 ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/ge

In [ ]:
# Test Gemini API

import google.generativeai as genai

GEMINI_API_KEY=""
genai.configure(api_key=GEMINI_API_KEY)

# 使用 Gemini 1.5 Pro 模型
# model = genai.GenerativeModel(model_name="gemini-1.5-pro")
model = genai.GenerativeModel(model_name="gemini-2.5-flash")


prompt = "請用繁體中文解釋什麼是量子纏結。"
response = model.generate_content(prompt)

print(response.text)


量子纏結（Quantum Entanglement）是量子力學中最奇特、最反直覺的現象之一，連愛因斯坦都曾稱之為「鬼魅般的超距作用」（spooky action at a distance）。

簡單來說，量子纏結是指當兩個或多個粒子（例如光子、電子等）以某種特殊方式關聯時，即使它們相隔遙遠，甚至在宇宙的兩端，它們的狀態依然是相互依存的，彷彿它們之間存在著某種看不見的瞬時連接。

**核心概念和特點：**

1.  **非局部關聯：**
    *   當粒子處於纏結狀態時，它們不再擁有獨立的、確定的屬性（例如自旋、極化、能量等）。
    *   相反，它們的屬性是相互關聯的，形成一個統一的整體。
    *   這種關聯是「非局部性」的，意味著它不受空間距離的限制。無論粒子相距多遠，它們之間的這種聯繫都是即時的。

2.  **狀態的不確定性與瞬時確定：**
    *   在被測量之前，纏結粒子本身的具體狀態是不確定的，它們處於一種「疊加態」中（即同時存在於多種可能狀態）。
    *   然而，一旦對其中一個纏結粒子進行測量，其結果會立即（瞬時）影響到另一個纏結粒子，使其狀態也隨之確定下來。
    *   例如，如果兩個纏結的光子，一個被測量到是垂直偏振，那麼即使另一個光子遠在天邊，你也會立刻知道它一定是水平偏振（如果它們被纏結為反向偏振）。

3.  **無法超光速通訊：**
    *   儘管這種影響是瞬時的，看似能夠傳遞資訊的速度超過光速，但它實際上**無法**用來實現超光速通訊。
    *   原因是發送方無法控制測量的結果（因為測量本身是隨機的），接收方也無法從中提取有意義的資訊。你只能確定另一個粒子的狀態，但無法選擇它會是哪種狀態。這就像你和朋友各有一雙纏結的襪子（一隻左，一隻右），當你看到自己的是左襪時，立即知道朋友的是右襪，但你無法決定自己看到的是左還是右，也無法透過這種方式向朋友發送訊息。

**一個比喻（雖然不完全精確，但有助於理解）：**

想像你有兩個特殊的硬幣，它們被「纏結」在一起。在你拋擲它們之前，你不知道每個硬幣是正面還是反面，甚至它們可能都不是確定的狀態。但它們之間存在一個規則：如果一個是正面，另一個就一定是反面。

現在，你和你的朋友各拿一個纏結的硬幣，你們相隔很遠。當你拋擲你手中的硬幣，假設它顯示的是「正面」，那麼即

In [ ]:
import google.generativeai as genai

# ✅ 设置 API key
GEMINI_API_KEY=""
genai.configure(api_key=GEMINI_API_KEY)

# ✅ 初始化模型
model = genai.GenerativeModel(model_name="gemini-2.5-flash")

# ✅ 评估函数
def evaluate_qa(question, gt_answer, pred_answer):
    prompt = f"""
你是一个严谨的评测专家，请判断以下问题的两个答案是否一致，并给出 1~5 分评分与简短评语：

【问题】
{question}

【标准答案】
{gt_answer}

【学生回答】
{pred_answer}

评分范围：
5 = 完全正确；
4 = 大致正确，略有差异；
3 = 一般正确，有部分缺漏；
2 = 有重大偏差；
1 = 完全错误。

請用如下格式回答：
分數: X
評語: Y
"""

    response = model.generate_content(prompt)
    try:
      content = response.text
    except ValueError:
        print("[模型未返回有效內容]")
        print("finish_reason:", response.candidates[0].finish_reason)
    return response.text


In [ ]:
# 测试评估函数
question = "什麼是量子糾纏？"
gt_answer = "量子糾纏是指兩個或多個粒子的量子態彼此關聯，即使它們距離遙遠，也能瞬間影響彼此。"
pred_answer = "量子糾纏是一種物理現象，兩個粒子會互相感應，即使在宇宙兩端也會同步改變。"

print(evaluate_qa(question, gt_answer, pred_answer))


In [ ]:
import time
import random

total_count, correct_count = 0, 0

for idx, row in df.iterrows():
  question = row['问题']
  gt_answer = row['答案'] # ground truth
  pred_answer = '' # prediction
  with open(f'./{PREFIX}_{idx + 1}.txt', 'r') as f:
    pred_answer = f.read().strip() # prediction

  time.sleep(random.uniform(1, 2))               # 1~2 秒随机延迟
  response = evaluate_qa(question, gt_answer, pred_answer)

  print(f'问题{idx + 1}：{question}')
  print(response)

  total_count += 1
  if "分數: 5" in response or "评分: 5" in response:
    correct_count += 1

print(f"\n✅ 完全正確的題目數：{correct_count}/{total_count}")

问题1：校歌為學校（包括小學、中學、大學等）宣告或者規定的代表該校的歌曲。用於體現該校的治學理念、辦學理想等學校文化。「虎山雄風飛揚」是哪間學校的校歌歌詞？
分數: 5
評語: 學生回答提供了完全正確的答案，精確指出了校歌歌詞所屬的學校。雖然表達方式與標準答案略有不同，但核心信息完全一致且正確無誤。
问题2：2025年初，NCC透過行政命令，規定民眾如果透過境外郵購無線鍵盤、滑鼠、藍芽耳機..等自用產品回台，每案一律加收審查費多少錢？
分數: 4
評語: 学生回答与标准答案的核心信息（750元）完全一致，且很好地复述了问题中的关键背景信息。唯一瑕疵是“鍑盘”的笔误，应为“键盘”，但这不影响对核心答案的判断，故评为大致正确。
问题3：第一代 iPhone 是由哪位蘋果 CEO 發表？
分數: 5
評語: 兩個答案都正確指出了第一代 iPhone 是由史蒂夫·乔布斯（Steve Jobs）發表。主要差異在於中文譯名的繁簡體（史提夫賈伯斯 vs 史蒂夫·乔布斯），以及學生回答中添加了信息來源說明，這些都不影響答案的正確性。兩者內容高度一致，可視為完全正確。
问题4：台灣大學進階英文免修申請規定中，托福網路測驗 TOEFL iBT 要達到多少分才能申請？
分數: 2
評語: 学生回答的主题和考试类型是正确的，但在具体分数要求上与标准答案存在明显差异。标准答案为72分，而学生回答为78分以上，这个分数上的偏差属于关键信息错误。
问题5：Rugby Union 中觸地 try 可得幾分？
分數: 5
評語: 学生的回答与标准答案在核心信息（触地try可得5分）上完全一致，表达准确无误。虽然学生回答中包含了“根據提供的資料”这一句，但这并未改变答案本身的正确性或与标准答案的一致性。
问题6：卑南族是位在臺東平原的一個原住民族，以驍勇善戰、擅長巫術聞名，曾經統治整個臺東平原。相傳卑南族的祖先發源自 ruvuwa'an，該地位於現今的哪個行政區劃？
分數: 3
評語: 學生答案正確指出了縣級行政區劃（臺東縣），但未能進一步提供更具體的鄉級行政區劃（太麻里鄉），與標準答案的完整性有部分缺漏。
问题7：熊信寬，藝名熊仔，是臺灣饒舌創作歌手。2022年獲得第33屆金曲獎最佳作詞人獎，2023年獲得第34屆金曲獎最佳華語專輯獎。請問熊仔的碩班指導教授為？
分數: 5
評語: 學生回答完整且正